# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
train_df = pd.read_csv(url, index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
train_df.shape

(6000, 61)

In [3]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular']

In [4]:
y.mean()

0.5

In [5]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [6]:
## Decision Tree Classifier
from sklearn.tree import  DecisionTreeClassifier

dtc = DecisionTreeClassifier( max_depth=None, random_state=43)
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)

from sklearn import metrics

print("Decision Tree Classifier")
print("Accuracy")
print(str(metrics.accuracy_score(y_pred, y_test)))
print(" ")
print("F1-Score")
print(str(metrics.f1_score(y_pred, y_test)))

Decision Tree Classifier
Accuracy
0.5446666666666666
 
F1-Score
0.5473823724320742


In [7]:
## Logistic Regression

from sklearn.linear_model import LogisticRegressionCV
lr = LogisticRegressionCV(cv = 5)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("Logistic Regression")
print("Accuracy")
print(str(metrics.accuracy_score(y_pred, y_test)))
print(" ")
print("F1-Score")
print(str(metrics.f1_score(y_pred, y_test)))

Logistic Regression
Accuracy
0.6166666666666667
 
F1-Score
0.6026261230131307


# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [8]:
import numpy as np

# set a seed for reproducibility
np.random.seed(42)

n_samples = X_train.shape[0]
n_B = 300

#  300 bagged samples (will be used to select rows from the Train set of the dataframe)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]
samples[0:10]


[array([ 860, 3772, 3092, ...,  320, 4059, 1131]),
 array([2578, 4042, 3367, ...,  341,  132,  264]),
 array([4188, 1685, 1434, ..., 1514, 4297,  633]),
 array([ 578, 3959,  272, ..., 1340, 3737, 1437]),
 array([  94, 3296,  948, ..., 2367, 3343, 3097]),
 array([1260, 1333, 4328, ..., 1082, 4247,  933]),
 array([2587, 4085,  557, ..., 2116, 2420,   54]),
 array([  84, 2888, 2779, ..., 1931, 1885, 2353]),
 array([3807, 2746,  580, ..., 2385, 1286,  925]),
 array([ 228, 1768, 2421, ...,  183,  837, 1892])]

In [9]:
from sklearn.tree import DecisionTreeClassifier

# grow each tree deep
treeclass = DecisionTreeClassifier(max_depth=None, random_state=42)

# DataFrame for storing predicted value from each tree
y_pred = pd.DataFrame(index=X_test.index, columns=[list(range(n_B))])

# grow one tree for each bootstrap sample and make predictions on testing data
for i, sample in enumerate(samples):
    X_train = X_train.iloc[sample]
    y_train = y_train.iloc[sample]
    treeclass.fit(X_train, y_train)
    y_pred[i] = treeclass.predict(X_test)


y_pred2 = (y_pred.sum(axis=1) >= (n_B / 2)).astype(np.int)

In [10]:
from sklearn import metrics
print("F1-Score")
print(metrics.f1_score(y_pred2, y_test))
print()
print("Accurancy")
print(metrics.accuracy_score(y_pred2, y_test))

F1-Score
0.5699680511182109

Accurancy
0.5513333333333333


In [11]:
from sklearn.tree import DecisionTreeClassifier

# grow each tree deep
treereg = DecisionTreeClassifier(max_depth=2, random_state=42)

# DataFrame for storing predicted value from each tree
y_pred_df = pd.DataFrame(index=X_test.index, columns=[list(range(n_B))])

# grow one tree for each bootstrap sample and make predictions on testing data
for i, sample in enumerate(samples):
    X_train = X_train.iloc[sample]
    y_train = y_train.iloc[sample]
    treereg.fit(X_train, y_train)
    y_pred[i] = treereg.predict(X_test)

y_pred2 = (y_pred.sum(axis=1) >= (n_B / 2)).astype(np.int)



In [12]:
from sklearn import metrics
print("F1-Score")
print(metrics.f1_score(y_pred2, y_test))
print()
print("Accurancy")
print(metrics.accuracy_score(y_pred2, y_test))

F1-Score
0.5994930291508239

Accurancy
0.5786666666666667


In [13]:
from sklearn.linear_model import LogisticRegression

# grow each tree deep
lr = LogisticRegression()

# DataFrame for storing predicted value from each tree
y_pred_df = pd.DataFrame(index=X_test.index, columns=[list(range(n_B))])

# grow one tree for each bootstrap sample and make predictions on testing data
for i, sample in enumerate(samples):
    X_train = X_train.iloc[sample]
    y_train = y_train.iloc[sample]
    lr.fit(X_train, y_train)
    y_pred[i] = lr.predict(X_test).astype(np.int)
    
y_pred2 = (y_pred.sum(axis=1) >= (n_B / 2)).astype(np.int)



In [14]:
from sklearn import metrics
print("F1-Score")
print(metrics.f1_score(y_pred2, y_test))
print()
print("Accurancy")
print(metrics.accuracy_score(y_pred2, y_test))

F1-Score
0.48852901484480427

Accurancy
0.49466666666666664


# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [15]:
#  BaggingClassifier to use DecisionTreeClassifier as the "base estimator"
from sklearn.ensemble import BaggingClassifier
bagclass= BaggingClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=100,bootstrap=True,
                          random_state=42, n_jobs=-1, oob_score=True)
bagclass.fit(X_train, y_train)
y_pred_DT2 = bagclass.predict(X_test)
print("BaggingClassifier with DecisionTreeClassifier and max_depth=2 as estimator ")
print("Accuracy")
print(str(metrics.accuracy_score(y_pred_DT2, y_test)))
print(" ")
print("F1-Score")
print(str(metrics.f1_score(y_pred_DT2, y_test)))
print(" ")

C:\Users\PACHO\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


BaggingClassifier with DecisionTreeClassifier and max_depth=2 as estimator 
Accuracy
0.576
 
F1-Score
0.4708818635607322
 


In [16]:
bagclass= BaggingClassifier(DecisionTreeClassifier(max_depth=None), n_estimators=100,bootstrap=True,
                          random_state=42, n_jobs=-1, oob_score=True)
bagclass.fit(X_train, y_train)
y_pred_DT0 = bagclass.predict(X_test)
print("BaggingClassifier with DecisionTreeClassifier as estimator ")
print("Accuracy")
print(str(metrics.accuracy_score(y_pred_DT0, y_test)))
print(" ")
print("F1-Score")
print(str(metrics.f1_score(y_pred_DT0, y_test)))
print(" ")

BaggingClassifier with DecisionTreeClassifier as estimator 
Accuracy
0.576
 
F1-Score
0.4708818635607322
 


In [17]:
from sklearn.linear_model import LogisticRegressionCV
bagclass= BaggingClassifier(LogisticRegressionCV(cv = 5), n_estimators=100,bootstrap=True,
                          random_state=42, n_jobs=-1, oob_score=True)
bagclass.fit(X_train, y_train)
y_pred_LR = bagclass.predict(X_test)
print("BaggingClassifier with Logistic Regression as estimator ")
print("Accuracy")
print(str(metrics.accuracy_score(y_pred_LR, y_test)))
print(" ")
print("F1-Score")
print(str(metrics.f1_score(y_pred_LR, y_test)))
print(" ")

BaggingClassifier with Logistic Regression as estimator 
Accuracy
0.498
 
F1-Score
0.5212968849332485
 


In [18]:
y_pred_ensabled = ((y_pred_DT2+y_pred_DT0+y_pred_LR)/3).astype(int)

In [19]:
print ("Essambled models metrics")
print("Accuracy")
print(str(metrics.accuracy_score(y_pred_ensabled, y_test)))
print("F1-Score")
print(str(metrics.f1_score(y_pred_ensabled, y_test)))


Essambled models metrics
Accuracy
0.5546666666666666
F1-Score
0.40674955595026646


In [20]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
models = {'LR': LogisticRegression(),
          'DT0': DecisionTreeClassifier(max_depth=None),
          'DT2': DecisionTreeClassifier(max_depth=2)}
y_pred_BR = pd.DataFrame (columns =models.keys(),index= y_test.index )
for i, model  in models.items():
    bagclass = BaggingClassifier(model, n_estimators=100,bootstrap=True, oob_score=True, 
                                 random_state=42,n_jobs=-1)
    bagclass.fit(X_train, y_train)
    y_pred_BR[i] = bagclass.predict(X_test)

y_pred_BR['predic'] = (y_pred_BR.sum(axis=1)/len(models)).astype(int)
print ("Bagging regressor models metrics")
print("Accuracy")
print(str(metrics.accuracy_score(y_pred_BR['predic'] , y_test)))
print("F1-Score")
print(str(metrics.f1_score(y_pred_BR['predic'] , y_test)))


Bagging regressor models metrics
Accuracy
0.548
F1-Score
0.38363636363636366


# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [22]:
import numpy as np


th=np.arange(0,1.1,0.1)
models = {'LR': LogisticRegression(),
          'DT0': DecisionTreeClassifier(max_depth=None),
          'DT2': DecisionTreeClassifier(max_depth=2)}
Scores_t = []
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for i in th:
    for model in models.keys():
        bagreg = BaggingClassifier(models[model], n_estimators=100,bootstrap=True, oob_score=True, random_state=42,n_jobs=-1)
        bagreg.fit(X_train, y_train)
        y_pred[model] = bagreg.predict_proba(X_test)
        y_pred[model] = y_pred[model].apply(lambda x: 1 if x>=i else 0)
        Scores_t.append([i,metrics.f1_score(y_test,y_pred[model]),metrics.accuracy_score(y_pred[model], y_test),model])
Scores_t=pd.DataFrame(Scores_t, columns=['Threshold','F1-Score','Accuracy','Model'])

In [23]:
Scores_t

,Threshold,F1-Score,Accuracy,Model
0,0.0,0.670213,0.504000,LR
1,0.0,0.670213,0.504000,DT0
2,0.0,0.670213,0.504000,DT2
3,0.1,0.566565,0.524667,LR
4,0.1,0.588409,0.441333,DT0
5,0.1,0.588409,0.441333,DT2
6,0.2,0.555347,0.526000,LR
7,0.2,0.586207,0.440000,DT0
8,0.2,0.586207,0.440000,DT2
9,0.3,0.546159,0.531333,LR


# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [24]:
models = {'LR': LogisticRegression(),
          'DT0': DecisionTreeClassifier(max_depth=None),
          'DT2': DecisionTreeClassifier(max_depth=2)}
metrics_t = []
mod2={}
for model in models.keys():
    mod2[model] = BaggingClassifier(models[model], n_estimators=100,bootstrap=True, oob_score=True, random_state=42,n_jobs=-1)
    mod2[model].fit(X_train, y_train)
    y_pred[model] = mod2[model].predict(X_test)
    metrics_t.append([metrics.f1_score(y_pred[model], y_test), metrics.accuracy_score(y_pred[model], y_test),model])
metrics_t=pd.DataFrame(metrics_t, columns=['F1-Score','Accurancy','Model'])

In [26]:
metrics_t=[]
for model in mod2.keys():
    errors = np.zeros(mod2[model].n_estimators)
    y_pred_all_ = np.zeros((X_test.shape[0], mod2[model].n_estimators))
    for i in range(mod2[model].n_estimators):
        oob_sample = ~mod2[model].estimators_samples_[i]
        y_pred_ = mod2[model].estimators_[i].predict(X_train.values[oob_sample])
        errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
        y_pred_all_[:, i] = mod2[model].estimators_[i].predict(X_test)
    alpha = (1 - errors) / (1 - errors).sum()
    y_pred[model] = (np.sum(y_pred_all_ * alpha, axis=1) >= 0.5).astype(np.int)
    metrics_t.append([metrics.f1_score(y_pred[model], y_test), metrics.accuracy_score(y_pred[model], y_test),model])

metrics_t=pd.DataFrame(metrics_t, columns=['F1 Score','Accurancy','Model'])
print('Best F1 Score:')
metrics_t.loc[metrics_t['F1 Score'].idxmax()]

C:\Users\PACHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
C:\Users\PACHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater_equal
  # This is added back by InteractiveShellApp.init_path()
C:\Users\PACHO\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Best F1 Score:


F1 Score     0.50889
Accurancy      0.466
Model             LR
Name: 0, dtype: object

# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [27]:
f2=[]
th=np.arange(0,1.1,0.1)
for j in th:
    for model in mod2.keys():
        errors = np.zeros(mod2[model].n_estimators)
        y_pred_all_ = np.zeros((X_test.shape[0], mod2[model].n_estimators))
        for i in range(mod2[model].n_estimators):
            oob_sample = ~mod2[model].estimators_samples_[i]
            y_pred_ = mod2[model].estimators_[i].predict(X_train.values[oob_sample])
            errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
            y_pred_all_[:, i] = mod2[model].estimators_[i].predict(X_test)
        alpha = (1 - errors) / (1 - errors).sum()
        y_pred[model] = (np.sum(y_pred_all_ * alpha, axis=1) >= j).astype(np.int)
        f2.append([j, metrics.f1_score(y_pred[model], y_test), metrics.accuracy_score(y_pred[model], y_test),model])
f2=pd.DataFrame(f2, columns=['Threshold','F1 Score','Accurancy','Model'])
print()
print('Best F1 Score:')
print(f2.loc[f2['F1 Score'].idxmax()])
print()

C:\Users\PACHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
C:\Users\PACHO\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in greater_equal
  del sys.path[0]
C:\Users\PACHO\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)



Best F1 Score:
Threshold           0
F1 Score     0.670213
Accurancy       0.504
Model              LR
Name: 0, dtype: object



# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score

In [28]:
ff=[]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
for model in mod2.keys():
    y_pred_all_ = np.zeros((X_test.shape[0], mod2[model].n_estimators))
    X_train_logit = np.zeros((X_train.shape[0], mod2[model].n_estimators))
    for i in range(mod2[model].n_estimators):
        X_train_logit[:, i] = mod2[model].estimators_[i].predict(X_train)
        y_pred_all_[:, i] = mod2[model].estimators_[i].predict(X_test)
    lr = LogisticRegressionCV(cv=5)
    lr.fit(X_train_logit, y_train)
    y_pred[model] = lr.predict(y_pred_all_)
    ff.append([model, metrics.f1_score(y_pred[model], y_test), metrics.accuracy_score(y_pred[model], y_test)])
ff=pd.DataFrame(ff, columns=['Model','F1 Score','Accurancy'])
print()
print('Best F1 Score:')
print(ff.loc[ff['F1 Score'].idxmax()])
print()


Best F1 Score:
Model             DT0
F1 Score     0.572966
Accurancy    0.576667
Name: 1, dtype: object

